In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud.bigquery import LoadJobConfig
from io import StringIO
import pandas as pd
import os
import datetime

pd.set_option('display.max_columns', None)

In [2]:
from google.cloud import bigquery

# Initialize a client
client = bigquery.Client()

# Define your query
query = """
SELECT 
    Date,
    TaggedPitchType,
    InducedVertBreak,
    PlateLocSide,
    PlateLocHeight,
    HorzBreak,
    runsHome, 
    Tilt,
    League,
    ExitSpeed,
    Direction,
    HitSpinRate,
    PositionAt110X,
    PositionAt110Y,
    manOnFirst, 
    manOnSecond, 
    manOnThird, 
    trackmanGameID, 
    BatterTeam,
    PitcherTeam,
    Pitcher,
    Batter,
    Balls,
    Outs,
    Strikes,
    RunsScored,
    Top_Bottom,
    Time, 
    PlayResult,
    Events,
    PitchCall,
    ExitSpeed,
    Angle,
    BatterSide,
    PitcherThrows,
    PitcherId,
    KorBB,
    RelSpeed,
    Extension,
    Tilt,
    RelSide,
    RelHeight,
    SpinRate,
    Inning 
    

"""

# Execute the query
query_job = client.query(query)

# Get the results as a pandas DataFrame
df = query_job.to_dataframe()


In [3]:

# Define the function to determine the value for 'Events'
def determine_event(row):
    if row['PlayResult'] != "Undefined":
        return row['PlayResult']
    elif row['KorBB'] in ["Strikeout", "Walk"]:
        return row['KorBB']
    else:
        return row['PitchCall']

# Apply the function to create the 'Events' column
df['Events'] = df.apply(determine_event, axis=1)

# Remove rows based on the values in 'Events' column
df = df[~df['Events'].isin([None, 'Slider', 'Sinker', 'Fastball', 'InPlay', 'Undefined', 'BattersInterference', 'FlyBall', 'GroundBall', 'CatchersInterfernece', 'CatchersInterference', 'Popup'])]

# Replace 'Homerun' with 'HomeRun' in the 'Events' column
df['Events'] = df['Events'].replace('Homerun', 'HomeRun')

# Replace 'Homerun' with 'HomeRun' in the 'Events' column
df['Events'] = df['Events'].replace('Fielderschoice', 'FieldersChoice')
# Print the unique values of the 'Events' column
print(df['Events'].unique())



['Single' 'FoulBall' 'Out' 'BallCalled' 'StrikeCalled' 'Strikeout'
 'StrikeSwinging' 'Walk' 'HitByPitch' 'Double' 'Error' 'BallinDirt'
 'HomeRun' 'FieldersChoice' 'StolenBase' 'Sacrifice' 'CaughtStealing'
 'Triple' 'BallIntentional']


In [4]:
groundball_count = (df['Events'] == 'CaughtStealing').sum()
print(f"There are {groundball_count} rows with 'GroundBall' in the 'Events' column.")


There are 542 rows with 'GroundBall' in the 'Events' column.


In [5]:
import pandas as pd

# Assuming df is loaded somewhere above this

# Convert manOnFirst, manOnSecond, and manOnThird to 1 for True and 0 for False
df['manOnFirst'] = df['manOnFirst'].astype(int)
df['manOnSecond'] = df['manOnSecond'].astype(int)
df['manOnThird'] = df['manOnThird'].astype(int)

# Compute runner_1b, runner_2b, and runner_3b columns
df['runner_1b'] = df['manOnFirst']
df['runner_2b'] = df['manOnSecond']
df['runner_3b'] = df['manOnThird']

# Create the runners column to represent the state of the bases
df['runners'] = df['runner_1b'].astype(str) + df['runner_2b'].astype(str) + df['runner_3b'].astype(str)

# Construct the half_inning column
df['half_inning'] = df['trackmanGameID'].astype(str) + ' ' + df['Inning'].astype(str) + ' ' + df['Top_Bottom'].astype(str)

# Calculate the runs_scored_play for each play using the runsHome column
df['runs_scored_play'] = df['RunsScored'].fillna(0)

# Aggregate runs scored per half inning
runs_scored = df.groupby('half_inning')['runs_scored_play'].agg(['min', 'sum'])
runs_scored.columns = ['runs_scored_play_min', 'runs_scored_inning_total']
runs_scored = runs_scored.reset_index()

# Merge the aggregated data back with the main dataframe
df = pd.merge(df, runs_scored, on='half_inning', how='left', suffixes=('', '_agg'))

# Ensure dataframe is sorted by half_inning and Time
df = df.sort_values(by=['half_inning', 'Time'])

# Calculate the cumulative runs scored up to (but not including) the current play
df['cumulative_runs_before_current'] = df.groupby('half_inning')['runs_scored_play'].cumsum() - df['runs_scored_play']

# Compute runs_scored_rest_of_inning, including runs from the current play
df['runs_scored_rest_of_inning'] = df['runs_scored_inning_total'] - df['cumulative_runs_before_current']

# Create the counts column by joining the balls and strikes columns with a dash
df['counts'] = df['Balls'].astype(str) + "-" + df['Strikes'].astype(str)

# Sort the dataframe by the 'Time' column
df_sorted = df.sort_values(by='Time')

# Filter the dataframe to only include rows with the specified counts
valid_counts = ['0-0', '1-0', '2-0', '3-0', '0-1', '1-1', '2-1', '3-1', '0-2', '1-2', '2-2', '3-2']
filtered_counts_df = df_sorted[df_sorted['counts'].isin(valid_counts)]

# Compute the RE288 table by grouping by the number of outs, runner situation, and the count
re_288 = filtered_counts_df[(filtered_counts_df['Outs'] >= 0) & (filtered_counts_df['Outs'] < 3)].groupby(['Outs', 'runners', 'counts'])['runs_scored_rest_of_inning'].mean().round(2).reset_index().rename(columns={'runs_scored_rest_of_inning': 're288'})



print("\nRE288 Table:")
print(re_288)



RE288 Table:
     Outs runners counts  re288
0       0     000    0-0   0.73
1       0     000    0-1   0.66
2       0     000    0-2   0.57
3       0     000    1-0    0.8
4       0     000    1-1   0.71
..    ...     ...    ...    ...
283     2     111    2-1   1.08
284     2     111    2-2   0.82
285     2     111    3-0   1.73
286     2     111    3-1   1.48
287     2     111    3-2   1.15

[288 rows x 4 columns]


In [6]:
# Assuming df and re_288 are loaded somewhere above this

# Merge only the 're288' column from the re_288 dataframe with the main df
df = pd.merge(df, re_288[['Outs', 'runners', 'counts', 're288']], on=['Outs', 'runners', 'counts'], how='left')

# Ensure dataframe is sorted by half_inning and Time
df = df.sort_values(by=['half_inning', 'Time'])

# Merge with the re_288 dataframe to get the run expectancy for the starting state of each play
df = df.merge(re_288, on=['Outs', 'runners', 'counts'], how='left', suffixes=('', '_start'))

# Shift the RE288 values to get the run expectancy for the end state of each play
df['re288_end'] = df.groupby('half_inning')['re288'].shift(-1).fillna(0)

# Calculate the RE288 change for each play
df['re288_change'] = (df['RunsScored'].fillna(0) + df['re288_end']) - df['re288_start']

# Group by the 'Events' column and calculate the average change for each event
event_avg_change_288 = df.groupby('Events')['re288_change'].mean()

# Display the average change for each event
print(event_avg_change_288)


Events
BallCalled         0.081131
BallIntentional   -0.001517
BallinDirt         0.115044
CaughtStealing    -0.554945
Double             0.848224
Error              0.588468
FieldersChoice    -0.392238
FoulBall          -0.064054
HitByPitch         0.456426
HomeRun             1.41507
Out               -0.377802
Sacrifice         -0.188812
Single             0.576945
StolenBase         0.193517
StrikeCalled      -0.089101
StrikeSwinging    -0.096922
Strikeout         -0.307994
Triple             1.146426
Walk               0.244287
Name: re288_change, dtype: Float64


In [7]:
# Merge the average run values for each event into the main dataframe
df = df.merge(event_avg_change_288, on='Events', how='left')

# Filter for rows where the pitch type is 'Slider'
slider_df = df[df['TaggedPitchType'] == 'Fastball']

# Calculate the total of the average run values for Slider pitches
total_avg_run_slider = slider_df['re288_change_x'].sum()

# Count the number of Slider pitches
count_slider = slider_df.shape[0]

# Calculate the average run value per Slider pitch
average_run_value_per_slider = total_avg_run_slider / count_slider if count_slider != 0 else 0

# Display the result
print(f"Average Run Value per Slider Pitch: {average_run_value_per_slider}")


Average Run Value per Slider Pitch: -0.0013285376250199254


In [8]:
# Filter for rows where the pitch type is 'Slider'
slider_df = df[df['TaggedPitchType'] == 'Slider']

# Calculate the total of the average run values for Slider pitches
total_avg_run_slider = slider_df['re288_change_x'].sum()

# Count the number of Slider pitches
count_slider = slider_df.shape[0]

# Calculate the average run value per Slider pitch
average_run_value_per_slider = total_avg_run_slider / count_slider if count_slider != 0 else 0

# Display the result
print(f"Average Run Value per Slider Pitch: {average_run_value_per_slider}")

Average Run Value per Slider Pitch: -0.0022243780838815757


In [9]:
# Calculate the average run value for each pitch type
avg_run_values_by_pitch = df.groupby('TaggedPitchType')['re288_change_x'].mean()

# Calculate the mean and standard deviation of these average run values
mean_run_value = avg_run_values_by_pitch.mean()
std_run_value = avg_run_values_by_pitch.std()

# Standardize the average run values for each pitch type
standardized_run_values = -(avg_run_values_by_pitch - mean_run_value) / std_run_value

# Rescale so the average is 100
A = 34  # Adjust as needed
rescaled_run_values = standardized_run_values * A + 100

# Display the rescaled average run values for each pitch type
print(rescaled_run_values.sort_values())



TaggedPitchType
Splitter             57.295747
Curveball            59.247157
ChangeUp             76.756506
TwoSeamFastBall      77.643026
Undefined            81.508692
Other                87.358299
FourSeamFastBall      98.65523
OneSeamFastBall     100.386314
Fastball            106.320582
Sinker              112.022454
Slider              112.576196
Cutter              161.858388
Knuckleball         168.371408
Name: re288_change_x, dtype: Float64


In [10]:
# Filtering the dataframe for pitcher 'Skenes, Paul'
df_skenes = df[df['Pitcher'] == 'Skenes, Paul']

# Calculating the average run value for each pitch type thrown by Skenes, Paul
avg_run_values_skenes = df_skenes.groupby('TaggedPitchType')['re288_change_x'].mean()

# Since the mean and standard deviation of average run values for all pitchers are needed,
# we compute them using the entire dataframe
mean_run_value = df.groupby('TaggedPitchType')['re288_change_x'].mean().mean()
std_run_value = df.groupby('TaggedPitchType')['re288_change_x'].mean().std()

# Standardize the average run values for each pitch type thrown by Skenes, Paul
standardized_run_values_skenes = -(avg_run_values_skenes - mean_run_value) / std_run_value

# Rescale so the average is 100 (using A = 34 as in the user's example)
A = 34
rescaled_run_values_skenes = standardized_run_values_skenes * A + 100

# Display the rescaled average run values for each pitch type thrown by Skenes, Paul
rescaled_run_values_skenes.sort_values()

TaggedPitchType
Sinker      -197.778158
Cutter      -127.397783
ChangeUp     -12.395568
Curveball    214.002454
Fastball     360.377481
Slider       374.041796
Splitter     568.370414
Name: re288_change_x, dtype: Float64

In [11]:
# Adding a count column for demonstration purposes
df['count'] = np.random.randint(50, 200, size=len(df))

# Filtering the dataframe for pitchers who have thrown at least 100 of each pitch type
df_filtered = df[df['count'] >= 100]

# Calculating the average run value for each pitch type for these pitchers
avg_run_values_filtered = df_filtered.groupby(['Pitcher', 'TaggedPitchType'])['re288_change_x'].mean()

# Standardizing these average run values
standardized_run_values_filtered = -(avg_run_values_filtered - mean_run_value) / std_run_value

# Getting the top 25 pitchers with the top average standardized run values for each TaggedPitchType
top_25_standardized_run_values = standardized_run_values_filtered.sort_values(ascending=False).head(25)

top_25_standardized_run_values

Pitcher              TaggedPitchType 
Reyes Jr., Miguel    ChangeUp            437.353646
Freeman, Miller      Fastball             418.87024
Camarillo, Frank     Curveball           414.762817
O'Brien, Caden       Curveball           355.205176
McGehee, Blake       Curveball           326.453212
Barfield, Brett      Curveball           326.453212
Carlos, Jonathan     Other               291.540113
Whooley, Michael     Fastball             277.16413
Grintz, Adam         Fastball            242.251031
Torres, Nicholas     Sinker              238.143608
Alba, Max            Sinker              223.767625
Kelly, Michael       Slider              221.713914
Georges, Grant       Curveball           212.472211
Barlow, Cam          Undefined            205.28422
Cunningham, Braedyn  Cutter              203.230508
Highfill, Sam        FourSeamFastBall    203.230508
Stromsborg, Tyler    Undefined           192.961949
Hutt, Colton         Slider              192.961949
Lobus, Taylor        Curve

In [12]:
# Standardize the 're288_change_x' column for the entire dataframe
mean_re288 = df['re288_change_x'].mean()
std_re288 = df['re288_change_x'].std()

# Creating the standardized column 'PV+' and rescaling it so the average is 100
A = 34
df['PV+'] = -(df['re288_change_x'] - mean_re288) / std_re288 * A + 100

# Displaying a snippet of the dataframe to verify the new column
df.head()

,Date,TaggedPitchType,InducedVertBreak,PlateLocSide,PlateLocHeight,HorzBreak,runsHome,Tilt,League,ExitSpeed,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,manOnFirst,manOnSecond,manOnThird,trackmanGameID,BatterTeam,PitcherTeam,Pitcher,Batter,Balls,Outs,Strikes,RunsScored,Top_Bottom,Time,PlayResult,Events,PitchCall,ExitSpeed_1,Angle,BatterSide,PitcherThrows,PitcherId,KorBB,RelSpeed,Extension,Tilt_1,RelSide,RelHeight,SpinRate,Inning,runner_1b,runner_2b,runner_3b,runners,half_inning,runs_scored_play,runs_scored_play_min,runs_scored_inning_total,cumulative_runs_before_current,runs_scored_rest_of_inning,counts,re288,re288_start,re288_end,re288_change_x,re288_change_y,count,PV+
0,2021-02-19,Fastball,15.20276,-1.10489,3.32767,-7.65449,<NA>,11:00,BW,NaN,NaN,NaN,NaN,NaN,0,0,0,20210219-CaesarUyesaka-1,SAN_GAU,SAN_BRO,"Grant II, Russell","Sprinkle, Jordan",0,0,0,0,Bottom,15:12:01.54,Undefined,StrikeCalled,StrikeCalled,NaN,NaN,Right,Left,1.000057e+09,Undefined,88.67889,5.77957,11:00,-2.46948,5.93924,2473.537666,1,0,0,0,000,20210219-CaesarUyesaka-1 1 Bottom,0,0,0,0,0,0-0,0.73,0.73,0.66,-0.07,-0.089101,50,107.556446
1,2021-02-19,Slider,-4.53040,-0.42461,2.28209,24.00206,<NA>,3:30,BW,NaN,NaN,NaN,NaN,NaN,0,0,0,20210219-CaesarUyesaka-1,SAN_GAU,SAN_BRO,"Grant II, Russell","Sprinkle, Jordan",0,0,1,0,Bottom,15:12:14.08,Undefined,StrikeCalled,StrikeCalled,NaN,NaN,Right,Left,1.000057e+09,Undefined,75.43488,4.88934,3:30,-2.41565,5.71442,2617.674525,1,0,0,0,000,20210219-CaesarUyesaka-1 1 Bottom,0,0,0,0,0,0-1,0.66,0.66,0.57,-0.09,-0.089101,134,109.733788
2,2021-02-19,Fastball,18.99151,-1.55110,3.06446,-4.31988,<NA>,11:30,BW,NaN,NaN,NaN,NaN,NaN,0,0,0,20210219-CaesarUyesaka-1,SAN_GAU,SAN_BRO,"Grant II, Russell","Sprinkle, Jordan",0,0,2,0,Bottom,15:12:30.34,Undefined,BallCalled,BallCalled,NaN,NaN,Right,Left,1.000057e+09,Undefined,89.61138,5.68231,11:30,-2.55893,5.91470,2509.297552,1,0,0,0,000,20210219-CaesarUyesaka-1 1 Bottom,0,0,0,0,0,0-2,0.57,0.57,0.59,0.02,0.081131,142,97.758409
3,2021-02-19,Fastball,17.23855,-1.70759,2.14130,-2.88087,<NA>,11:45,BW,NaN,NaN,NaN,NaN,NaN,0,0,0,20210219-CaesarUyesaka-1,SAN_GAU,SAN_BRO,"Grant II, Russell","Sprinkle, Jordan",1,0,2,0,Bottom,15:12:45.17,Undefined,BallCalled,BallCalled,NaN,NaN,Right,Left,1.000057e+09,Undefined,90.14478,5.64494,11:45,-2.41975,6.00118,2544.914900,1,0,0,0,000,20210219-CaesarUyesaka-1 1 Bottom,0,0,0,0,0,1-2,0.59,0.59,0.68,0.09,0.081131,113,90.137714
4,2021-02-19,Fastball,19.91426,-0.75633,3.35093,-3.63071,<NA>,11:45,BW,NaN,NaN,NaN,NaN,NaN,0,0,0,20210219-CaesarUyesaka-1,SAN_GAU,SAN_BRO,"Grant II, Russell","Sprinkle, Jordan",2,0,2,0,Bottom,15:13:01.50,Undefined,Strikeout,StrikeSwinging,NaN,NaN,Right,Left,1.000057e+09,Strikeout,89.08073,5.87130,11:45,-2.37613,5.94212,2520.207202,1,0,0,0,000,20210219-CaesarUyesaka-1 1 Bottom,0,0,0,0,0,2-2,0.68,0.68,0.39,-0.29,-0.307994,60,131.507203


In [15]:
# Filtering the dataframe for the year 2023
df_2023 = df[pd.to_datetime(df['Date']).dt.year == 2023]

# Grouping by 'Pitcher' and 'PitcherTeam', calculating the average 'PV+' for each pitcher in 2023
pitcher_pv_avg_2023 = df_2023.groupby(['Pitcher', 'PitcherTeam'])['PV+'].mean()

# Adding a column for the total number of pitches thrown by each pitcher in 2023
pitcher_pitch_count_2023 = df_2023.groupby(['Pitcher', 'PitcherTeam']).size()

# Combining the average PV+ and pitch count into a single dataframe for 2023
pitcher_summary_2023 = pd.DataFrame({
    'Average PV+': pitcher_pv_avg_2023,
    'Total Pitches': pitcher_pitch_count_2023
})

# Filtering pitchers who have thrown at least 200 pitches in 2023
filtered_pitchers_2023 = pitcher_summary_2023[pitcher_summary_2023['Total Pitches'] >= 200]

# Sorting the pitchers by their average PV+ in 2023 and getting the top 25
top_25_pitchers_2023 = filtered_pitchers_2023.sort_values(by='Average PV+', ascending=False).head(25)

top_25_pitchers_2023

,,Average PV+,Total Pitches
Pitcher,PitcherTeam,,
"Russell, AJ",TEN_VOL,106.111365,336
"Proctor, Nick",SOU_GAM,105.634706,230
"Rogers, Grant",MCN_COW,105.626226,492
"Parks, Michael",EVA_ACE,105.052036,233
"Leach, Hudson",MIA_RED,104.765769,213
"Mahlstedt, Lucas",WOF_TER,104.741191,413
"Harrison, Charles",UCLA,104.702088,320
"Wilson, Jacob",CAL_LAN,104.674385,224
"Gearing, Chase",NIC_COL,104.474587,201


In [16]:
# Checking if the 'Events' column in df matches the indices in event_avg_change_288
events_in_df = df['Events'].unique()
events_in_avg_change = event_avg_change_288.index

# Checking for the presence of each event from event_avg_change_288 in the df dataframe
events_presence = {event: event in events_in_df for event in events_in_avg_change}

events_presence

{'BallCalled': True,
 'BallIntentional': True,
 'BallinDirt': True,
 'CaughtStealing': True,
 'Double': True,
 'Error': True,
 'FieldersChoice': True,
 'FoulBall': True,
 'HitByPitch': True,
 'HomeRun': True,
 'Out': True,
 'Sacrifice': True,
 'Single': True,
 'StolenBase': True,
 'StrikeCalled': True,
 'StrikeSwinging': True,
 'Strikeout': True,
 'Triple': True,
 'Walk': True}

In [19]:
# Standardizing the event_avg_change_288 values
mean_event_change = event_avg_change_288.mean()
std_event_change = event_avg_change_288.std()

# Creating a standardized and rescaled series from event_avg_change_288
A = 34
standardized_event_change = -(event_avg_change_288 - mean_event_change) / std_event_change * A + 100

# Mapping the standardized values to the 'Events' column in the df dataframe
df['PVplus'] = df['Events'].map(standardized_event_change)

# Displaying a snippet of the dataframe to verify the new column
print(df.head())

         Date TaggedPitchType  InducedVertBreak  PlateLocSide  PlateLocHeight  \
0  2021-02-19        Fastball          15.20276      -1.10489         3.32767   
1  2021-02-19          Slider          -4.53040      -0.42461         2.28209   
2  2021-02-19        Fastball          18.99151      -1.55110         3.06446   
3  2021-02-19        Fastball          17.23855      -1.70759         2.14130   
4  2021-02-19        Fastball          19.91426      -0.75633         3.35093   

   HorzBreak  runsHome   Tilt League  ExitSpeed  Direction  HitSpinRate  \
0   -7.65449      <NA>  11:00     BW        NaN        NaN          NaN   
1   24.00206      <NA>   3:30     BW        NaN        NaN          NaN   
2   -4.31988      <NA>  11:30     BW        NaN        NaN          NaN   
3   -2.88087      <NA>  11:45     BW        NaN        NaN          NaN   
4   -3.63071      <NA>  11:45     BW        NaN        NaN          NaN   

   PositionAt110X  PositionAt110Y  manOnFirst  manOnSecond  ma

In [20]:
# Filtering the dataframe for the year 2023
df_2023 = df[pd.to_datetime(df['Date']).dt.year == 2023]

# Grouping by 'Pitcher' and 'PitcherTeam', calculating the average 'PV+' for each pitcher in 2023
pitcher_pv_avg_2023 = df_2023.groupby(['Pitcher', 'PitcherTeam'])['PVplus'].mean()

# Adding a column for the total number of pitches thrown by each pitcher in 2023
pitcher_pitch_count_2023 = df_2023.groupby(['Pitcher', 'PitcherTeam']).size()

# Combining the average PV+ and pitch count into a single dataframe for 2023
pitcher_summary_2023 = pd.DataFrame({
    'Average PVplus': pitcher_pv_avg_2023,
    'Total Pitches': pitcher_pitch_count_2023
})

# Filtering pitchers who have thrown at least 200 pitches in 2023
filtered_pitchers_2023 = pitcher_summary_2023[pitcher_summary_2023['Total Pitches'] >= 200]

# Sorting the pitchers by their average PV+ in 2023 and getting the top 25
top_25_pitchers_2023 = filtered_pitchers_2023.sort_values(by='Average PVplus', ascending=False).head(25)

top_25_pitchers_2023

,,Average PVplus,Total Pitches
Pitcher,PitcherTeam,,
"Russell, AJ",TEN_VOL,115.836023,336
"Rogers, Grant",MCN_COW,115.592601,492
"Prichard, Tyler",ARM_BLA,115.251725,283
"Proctor, Nick",SOU_GAM,115.061657,230
"Takacs, Noah",SAC_HOR,114.996121,215
"Leach, Hudson",MIA_RED,114.97391,213
"Storm, Justin",SOU_GOL,114.944172,428
"Murray, Justin",HOU_COU,114.893811,349
"Georgini, Joe",NJI_HIG,114.877605,243


In [21]:
# Filtering the dataframe for 'Skenes, Paul'
df_skenes = df[df['Pitcher'] == 'Skenes, Paul']

# Counting the number of each pitch type thrown by 'Skenes, Paul'
skenes_pitch_counts = df_skenes['TaggedPitchType'].value_counts()

skenes_pitch_counts

TaggedPitchType
Fastball     940
Slider       333
ChangeUp     104
Curveball     40
Sinker        27
Cutter        14
Splitter       4
Name: count, dtype: int64

In [23]:
# Filtering the dataframe for the year 2023
df_2023 = df[pd.to_datetime(df['Date']).dt.year == 2023]

# Counting the number of each pitch type thrown by each pitcher in 2023
pitch_type_count_2023 = df_2023.groupby(['Pitcher', 'TaggedPitchType']).size().reset_index(name='PitchCount')

# Filtering out pitch types where a pitcher has thrown less than 50 of that type in 2023
filtered_pitch_types_2023 = pitch_type_count_2023[pitch_type_count_2023['PitchCount'] >= 50]

# Merging the filtered pitch types with the main df_2023 dataframe
df_2023_filtered = df_2023.merge(filtered_pitch_types_2023[['Pitcher', 'TaggedPitchType']], on=['Pitcher', 'TaggedPitchType'], how='inner')

# Grouping by 'Pitcher' and 'PitcherTeam', calculating the average 'PVplus' for each pitcher in 2023
pitcher_pv_avg_2023_filtered = df_2023_filtered.groupby(['Pitcher', 'PitcherTeam'])['PVplus'].mean()

# Adding a column for the total number of pitches thrown by each pitcher in 2023
pitcher_pitch_count_2023_filtered = df_2023_filtered.groupby(['Pitcher', 'PitcherTeam']).size()

# Combining the average PV+ and pitch count into a single dataframe for 2023
pitcher_summary_2023_filtered = pd.DataFrame({
    'Average PVplus': pitcher_pv_avg_2023_filtered,
    'Total Pitches': pitcher_pitch_count_2023_filtered
})

# Filtering pitchers who have thrown at least 200 pitches in 2023
filtered_pitchers_2023_filtered = pitcher_summary_2023_filtered[pitcher_summary_2023_filtered['Total Pitches'] >= 250]

# Sorting the pitchers by their average PV+ in 2023 and getting the top 25
top_25_pitchers_2023_filtered = filtered_pitchers_2023_filtered.sort_values(by='Average PVplus', ascending=False).head(25)

top_25_pitchers_2023_filtered



,,Average PVplus,Total Pitches
Pitcher,PitcherTeam,,
"Russell, AJ",TEN_VOL,115.962819,327
"Maldonado, Nick",VAN_COM,115.817996,302
"Rogers, Grant",MCN_COW,115.744487,446
"Murray, Justin",HOU_COU,115.514101,258
"Carlson, Cade",ETS_BUC,115.283233,251
"Lowder, Rhett",WAK_DEA,114.911345,1002
"Storm, Justin",SOU_GOL,114.829819,414
"Roland, Cole",WAK_DEA,114.714785,395
"Skenes, Paul",LSU_TIG,114.707347,1285


In [29]:
# Filtering the dataframe for the year 2023
df_2023 = df[pd.to_datetime(df['Date']).dt.year == 2023]

# Grouping by 'TaggedPitchType', calculating the average 'PVplus' for each pitch type in 2023
pitch_type_pv_avg_2023 = df_2023.groupby(['TaggedPitchType', 'Pitcher'])['PVplus'].mean()

# Adding a column for the total number of pitches of each type thrown in 2023
pitch_type_pitch_count_2023 = df_2023.groupby(['TaggedPitchType', 'Pitcher']).size()

# Combining the average PVplus and pitch count into a single dataframe for 2023
pitch_type_summary_2023 = pd.DataFrame({
    'Average PVplus': pitch_type_pv_avg_2023,
    'Total Pitches': pitch_type_pitch_count_2023
})

# Filtering pitch types that have been thrown at least 200 times in 2023
filtered_pitch_types_2023 = pitch_type_summary_2023[pitch_type_summary_2023['Total Pitches'] >= 200]

# Sorting the pitch types by their average PVplus in 2023
top_pitch_types_2023 = filtered_pitch_types_2023.sort_values(by='Average PVplus', ascending=False)

top_pitch_types_2023

Average PVplus  Total Pitches
TaggedPitchType Pitcher                                               
Slider          Tolle, Payton                  117.4024            216
                Burns, Mason                 116.053369            236
ChangeUp        Lowder, Rhett                115.939042            227
Slider          Skenes, Paul                 115.923003            306
                Christophersen, William       115.91611            221
...                                                 ...            ...
Fastball        Miller-Green, Lyle           108.725555            213
                Williams, Dalton             108.719042            216
                Lee, Justin                  108.371878            227
                Murdock, Mason               107.994672            211
                Aycock, Grant                107.987492            232

[1017 rows x 2 columns]